In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
data_path = '../data/'
df = pd.read_csv(os.path.join(data_path, 'WA_Fn-UseC_-HR-Employee-Attrition.csv'))
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


There are 35 features in total in the dataset about. 
Let's focus on a few of them:
- Age
- Attrition
- Department
- DistanceFromHome
- Education
- EnvironmentSatisfaction
- Gender
- JobSatisfaction
- MaritalStatus
- MonthlyIncome
- OverTime
- PerformanceRating
- RelationshipSatisfaction
- TotalWorkingYears
- YearsAtCompany

In this lab, we will use attrition as our label, to try to predict the attrition status accroding to other attributes. 


In [3]:
to_keep = {'Age', 'Attrition', 'Department','DistanceFromHome', 'Education', 'EnvironmentSatisfaction', 'Gender', 'JobSatisfaction', 'MaritalStatus',
           'MonthlyIncome', 'OverTime', 'PerformanceRating', 'RelationshipSatisfaction','TotalWorkingYears','YearsAtCompany'}
to_drop = set(df.columns)-to_keep
df.drop(to_drop, axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 15 columns):
Age                         1470 non-null int64
Attrition                   1470 non-null object
Department                  1470 non-null object
DistanceFromHome            1470 non-null int64
Education                   1470 non-null int64
EnvironmentSatisfaction     1470 non-null int64
Gender                      1470 non-null object
JobSatisfaction             1470 non-null int64
MaritalStatus               1470 non-null object
MonthlyIncome               1470 non-null int64
OverTime                    1470 non-null object
PerformanceRating           1470 non-null int64
RelationshipSatisfaction    1470 non-null int64
TotalWorkingYears           1470 non-null int64
YearsAtCompany              1470 non-null int64
dtypes: int64(10), object(5)
memory usage: 172.3+ KB


# Preprocessing

It's good that we don't have any null value. Let's one hot encode the Attrition, Department, Gender, MaritalStatus and Overtime. 

In [4]:
to_convert = ['Education','EnvironmentSatisfaction','JobSatisfaction',
            'PerformanceRating','RelationshipSatisfaction']
for col in to_convert:
    df[col] = df[col].astype(np.str)
    

In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

to_encode = {'Attrition', 'Department','Gender','MaritalStatus','OverTime','Education','EnvironmentSatisfaction','JobSatisfaction',
            'PerformanceRating','RelationshipSatisfaction'}
encoders = dict()

for col in to_encode:
    if col=="attrition":
        tmp = LabelEncoder()
        df[col] = tmp.fit_transform(df[col])
    else:
        encoders[col] = LabelEncoder()
        df[col+'_int'] = encoders[col].fit_transform(df[col])
    

Then, let's scale the numeric features. 

In [6]:
categorical_features =list(to_encode)
categorical_features = [x+'_int' for x in categorical_features]
numerics = set(df.columns) - to_encode
numerics = list(numerics - set(categorical_features))

for atr in numerics:
    df[atr] = df[atr].astype(np.float)    
    ss = StandardScaler()
    df[atr] = ss.fit_transform(df[atr].values.reshape(-1, 1))
    
feature_columns = categorical_features + numerics

In [7]:
numerics

['DistanceFromHome',
 'TotalWorkingYears',
 'Age',
 'YearsAtCompany',
 'MonthlyIncome']

In [8]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 25 columns):
Age                             1470 non-null float64
Attrition                       1470 non-null object
Department                      1470 non-null object
DistanceFromHome                1470 non-null float64
Education                       1470 non-null object
EnvironmentSatisfaction         1470 non-null object
Gender                          1470 non-null object
JobSatisfaction                 1470 non-null object
MaritalStatus                   1470 non-null object
MonthlyIncome                   1470 non-null float64
OverTime                        1470 non-null object
PerformanceRating               1470 non-null object
RelationshipSatisfaction        1470 non-null object
TotalWorkingYears               1470 non-null float64
YearsAtCompany                  1470 non-null float64
Gender_int                      1470 non-null int64
Education_int                   1470 non-

## Model Selection

In [61]:
# taken from https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/
# this uses the Keras Wrapper to make the model usable by sk-learn

# Use scikit-learn to grid search the batch size and epochs
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

# Function to create model, required for KerasClassifier
def create_model(num_neurons=12, input_dim=8):
    # create model
    model = Sequential()
    
    # num_neurons is a list of the number nuerons at each layer
    for layer, num in enumerate(num_neurons):
        if layer == 0:
            model.add(Dense(num, input_dim=input_dim, activation='relu'))
        else:
            model.add(Dense(units=num, activation='relu'))
    
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

Now that we can creat models in that can be used by SKLearn, lets GridSearch over the parameters we identified.  Note that the number of neurons and number of layers is combined into one parameter called `num_neurons` which is a list of the number of output nerouns at each layer.

In [9]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation, Input
from keras.layers import Embedding, Flatten, Merge, concatenate
from keras.models import Model
from sklearn.preprocessing import OneHotEncoder
from sklearn import metrics as mt

# stratified 90/10 train/test split`
df_train, df_test = train_test_split(df, test_size=0.1, stratify=df.Attrition)

X_train = ss.fit_transform(df_train[feature_columns].values).astype(np.float32)
X_test = ss.fit_transform(df_test[feature_columns].values).astype(np.float32)

y_train = df_train['Attrition_int'].values.astype(np.int)
y_test = df_test['Attrition_int'].values.astype(np.int)

print('train', X_train.shape, 'test', X_test.shape)

Using TensorFlow backend.


train (1323, 15) test (147, 15)


In [57]:
cross_columns = [['Gender','MaritalStatus'],
                    ['Education', 'JobSatisfaction'],['Department','PerformanceRating'],
                    ['Education', 'JobSatisfaction','RelationshipSatisfaction'],['Department','OverTime'],
                ]

In [58]:
ohe = OneHotEncoder()
X_train_ohe = ohe.fit_transform(df_train[categorical_features].values)
X_test_ohe = ohe.transform(df_test[categorical_features].values)


X_train_num =  df_train[numerics].values
X_test_num = df_test[numerics].values

In [59]:
# we need to create separate sequential models for each embedding
embed_branches = []
X_ints_train = []
X_ints_test = []
all_inputs = []
all_branch_outputs = []


# reset this input branch
all_branch_outputs = []
# add in the embeddings
for col in categorical_features:
    # encode as ints for the embedding
    X_ints_train.append( df_train[col].values )
    X_ints_test.append( df_test[col].values )
    
    # get the number of categories
    N = max(X_ints_train[-1]+1) # same as the max(df_train[col])
    
    # create embedding branch from the number of categories
    inputs = Input(shape=(1,),dtype='int32', name=col)
    all_inputs.append(inputs)
    x = Embedding(input_dim=N, output_dim=int(np.sqrt(N)), input_length=1)(inputs)
    x = Flatten()(x)
    all_branch_outputs.append(x)
    
# also get a dense branch of the numeric features
all_inputs.append(Input(shape=(X_train_num.shape[1],),sparse=False,name='numeric_data'))
x = Dense(units=20, activation='relu')(all_inputs[-1])
all_branch_outputs.append( x )

# merge the branches together
deep_branch = concatenate(all_branch_outputs)
    
final_branch = Dense(units=1,activation='sigmoid')(deep_branch)

model = Model(inputs=all_inputs, outputs=final_branch)

model.compile(optimizer='adagrad',
              loss='mean_squared_error',
              metrics=['accuracy'])

model.fit(X_ints_train+ [X_train_num],
        y_train, epochs=10, batch_size=32, verbose=0)

In [60]:
model2 = Model(inputs=model.inputs, outputs=model.get_layer(index=-2).output)

X_embed = model2.predict(X_ints_train+ [X_train_num])

In [62]:
# this will move inside nested CV
from sklearn.model_selection import GridSearchCV
num_neurons = [[5, 10], [5, 10, 20]]
class_weight=class_weight = [{0:x, 1:1-x} for x in np.linspace(0.1, 0.5, 2)]
param_grid = dict(num_neurons=num_neurons,
                  class_weight=class_weight)

model = KerasClassifier(build_fn=create_model, input_dim=X_embed.shape[-1], epochs=10, verbose=0)
g = GridSearchCV(estimator=model, param_grid=param_grid, verbose=3, scoring='recall')
r = g.fit(X_embed, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] class_weight={0: 0.10000000000000001, 1: 0.90000000000000002}, num_neurons=[5, 10] 
[CV]  class_weight={0: 0.10000000000000001, 1: 0.90000000000000002}, num_neurons=[5, 10], score=0.987012987012987, total=   2.2s
[CV] class_weight={0: 0.10000000000000001, 1: 0.90000000000000002}, num_neurons=[5, 10] 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.2s remaining:    0.0s


[CV]  class_weight={0: 0.10000000000000001, 1: 0.90000000000000002}, num_neurons=[5, 10], score=0.9838709677419355, total=   2.7s
[CV] class_weight={0: 0.10000000000000001, 1: 0.90000000000000002}, num_neurons=[5, 10] 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    5.1s remaining:    0.0s


[CV]  class_weight={0: 0.10000000000000001, 1: 0.90000000000000002}, num_neurons=[5, 10], score=0.9459459459459459, total=   2.3s
[CV] class_weight={0: 0.10000000000000001, 1: 0.90000000000000002}, num_neurons=[5, 10, 20] 
[CV]  class_weight={0: 0.10000000000000001, 1: 0.90000000000000002}, num_neurons=[5, 10, 20], score=1.0, total=   3.9s
[CV] class_weight={0: 0.10000000000000001, 1: 0.90000000000000002}, num_neurons=[5, 10, 20] 
[CV]  class_weight={0: 0.10000000000000001, 1: 0.90000000000000002}, num_neurons=[5, 10, 20], score=1.0, total=   2.4s
[CV] class_weight={0: 0.10000000000000001, 1: 0.90000000000000002}, num_neurons=[5, 10, 20] 
[CV]  class_weight={0: 0.10000000000000001, 1: 0.90000000000000002}, num_neurons=[5, 10, 20], score=1.0, total=   2.6s
[CV] class_weight={0: 0.5, 1: 0.5}, num_neurons=[5, 10] ..............
[CV]  class_weight={0: 0.5, 1: 0.5}, num_neurons=[5, 10], score=0.24675324675324675, total=   2.6s
[CV] class_weight={0: 0.5, 1: 0.5}, num_neurons=[5, 10] ........

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:   37.2s finished


In [66]:
best_deep = r.best_estimator_.model
print(best_deep.summary())
best_deep_params = r.best_estimator_.get_params()
print(best_deep_params)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_193 (Dense)            (None, 5)                 175       
_________________________________________________________________
dense_194 (Dense)            (None, 10)                60        
_________________________________________________________________
dense_195 (Dense)            (None, 20)                220       
_________________________________________________________________
dense_196 (Dense)            (None, 1)                 21        
Total params: 476
Trainable params: 476
Non-trainable params: 0
_________________________________________________________________
None
{'input_dim': 34, 'epochs': 10, 'verbose': 0, 'class_weight': {0: 0.10000000000000001, 1: 0.90000000000000002}, 'num_neurons': [5, 10, 20], 'build_fn': <function create_model at 0x000002CE87521D90>}


In [65]:

# we need to create separate sequential models for each embedding
embed_branches = []
X_ints_train = []
X_ints_test = []
all_inputs = []
all_branch_outputs = []

for cols in cross_columns:
    # encode crossed columns as ints for the embedding
    enc = LabelEncoder()
    
    # create crossed labels
    # needs to be commented better, Eric!
    X_crossed_train = df_train[cols].apply(lambda x: '_'.join(x), axis=1)
    X_crossed_test = df_test[cols].apply(lambda x: '_'.join(x), axis=1)
    
    enc.fit(np.hstack((X_crossed_train.values,  X_crossed_test.values)))
    X_crossed_train = enc.transform(X_crossed_train)
    X_crossed_test = enc.transform(X_crossed_test)
    X_ints_train.append( X_crossed_train )
    X_ints_test.append( X_crossed_test )
    
    # get the number of categories
    N = max(X_ints_train[-1]+1) # same as the max(df_train[col])
    
    # create embedding branch from the number of categories
    inputs = Input(shape=(1,),dtype='int32', name = '_'.join(cols))
    all_inputs.append(inputs)
    x = Embedding(input_dim=N, output_dim=int(np.sqrt(N)), input_length=1)(inputs)
    x = Flatten()(x)
    all_branch_outputs.append(x)
    
# merge the branches together
wide_branch = concatenate(all_branch_outputs)

# reset this input branch
all_branch_outputs = []
# add in the embeddings
for col in categorical_features:
    # encode as ints for the embedding
    X_ints_train.append( df_train[col].values )
    X_ints_test.append( df_test[col].values )
    
    # get the number of categories
    N = max(X_ints_train[-1]+1) # same as the max(df_train[col])
    
    # create embedding branch from the number of categories
    inputs = Input(shape=(1,),dtype='int32', name=col)
    all_inputs.append(inputs)
    x = Embedding(input_dim=N, output_dim=int(np.sqrt(N)), input_length=1)(inputs)
    x = Flatten()(x)
    all_branch_outputs.append(x)
    
# also get a dense branch of the numeric features
all_inputs.append(Input(shape=(X_train_num.shape[1],),sparse=False,name='numeric_data'))
x = Dense(units=20, activation='relu')(all_inputs[-1])
all_branch_outputs.append( x )

# merge the branches together
deep_branch = concatenate(all_branch_outputs)

# here is where we'll use the result of the GridSearch
for layer in best_deep.layers[:-1]:
    deep_branch = layer(deep_branch)
    
final_branch = concatenate([wide_branch, deep_branch])
final_branch = Dense(units=1,activation='sigmoid')(final_branch)

model = Model(inputs=all_inputs, outputs=final_branch)

model.compile(optimizer='adagrad',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_ints_train+ [X_train_num],
        y_train, epochs=10, batch_size=32, verbose=1, class_weight=best_deep_params['class_weight'])

Epoch 1/10
1323/1323 [==============================] - 1s - loss: 0.1246 - acc: 0.3583     
Epoch 2/10
1323/1323 [==============================] - 0s - loss: 0.0725 - acc: 0.8670     
Epoch 3/10
1323/1323 [==============================] - 0s - loss: 0.0362 - acc: 0.9902     
Epoch 4/10
1323/1323 [==============================] - 0s - loss: 0.0194 - acc: 0.9985     
Epoch 5/10
1323/1323 [==============================] - 0s - loss: 0.0118 - acc: 1.0000     
Epoch 6/10
1323/1323 [==============================] - 0s - loss: 0.0078 - acc: 1.0000     
Epoch 7/10
1323/1323 [==============================] - 0s - loss: 0.0055 - acc: 1.0000     
Epoch 8/10
1323/1323 [==============================] - 0s - loss: 0.0041 - acc: 1.0000     
Epoch 9/10
1323/1323 [==============================] - 0s - loss: 0.0032 - acc: 1.0000     
Epoch 10/10
1323/1323 [==============================] - 0s - loss: 0.0026 - acc: 1.0000     


In [56]:
yhat = np.round(model.predict(X_ints_test + [X_test_num]))
print(mt.confusion_matrix(y_test,yhat),mt.recall_score(y_test,yhat))

[[123   0]
 [  0  24]] 1.0


In [ ]:
from sklearn.model_selection import StratifiedKFold

outer_loop = StratifiedKFold(n_splits=5)

X = df.drop('Attrition', axis=1).values
y = df.Attrition.values

scores = []

for train_idx, test_idx in outer_loop.split(X, y):
    
    # split data
    df_train = pd.DataFrame(X[train_idx], columns=df.columns.drop('Attrition'))
    df_test = pd.DataFrame(X[test_idx], columns=df.columns.drop('Attrition'))
    
    X_train = ss.fit_transform(df_train[feature_columns].values).astype(np.float32)
    X_test = ss.fit_transform(df_test[feature_columns].values).astype(np.float32)

    y_train = df_train['Attrition_int'].values.astype(np.int)
    y_test = df_test['Attrition_int'].values.astype(np.int)

    # the whole thing
    X_train_num =  df_train[numerics].values
    X_test_num = df_test[numerics].values

    
    # train deep
    print('training embedings')
    # we need to create separate sequential models for each embedding
    embed_branches = []
    X_ints_train = []
    X_ints_test = []
    all_inputs = []
    all_branch_outputs = []


    # reset this input branch
    all_branch_outputs = []
    # add in the embeddings
    for col in categorical_features:
        # encode as ints for the embedding
        X_ints_train.append( df_train[col].values )
        X_ints_test.append( df_test[col].values )

        # get the number of categories
        N = max(X_ints_train[-1]+1) # same as the max(df_train[col])

        # create embedding branch from the number of categories
        inputs = Input(shape=(1,),dtype='int32', name=col)
        all_inputs.append(inputs)
        x = Embedding(input_dim=N, output_dim=int(np.sqrt(N)), input_length=1)(inputs)
        x = Flatten()(x)
        all_branch_outputs.append(x)

    # also get a dense branch of the numeric features
    all_inputs.append(Input(shape=(X_train_num.shape[1],),sparse=False,name='numeric_data'))
    x = Dense(units=20, activation='relu')(all_inputs[-1])
    all_branch_outputs.append( x )

    # merge the branches together
    deep_branch = concatenate(all_branch_outputs)

    final_branch = Dense(units=1,activation='sigmoid')(deep_branch)

    model = Model(inputs=all_inputs, outputs=final_branch)

    model.compile(optimizer='adagrad',
                  loss='mean_squared_error',
                  metrics=['accuracy'])

    model.fit(X_ints_train+ [X_train_num],
            y_train, epochs=10, batch_size=32, verbose=1)
    
    
    # get output from embedding
    model2 = Model(inputs=model.inputs, outputs=model.get_layer(index=-2).output)

    X_embed = model2.predict(X_ints_train+ [X_train_num])
    
    # Grid Search Deep
    print('Grid Searching the layers of the Deep')
    from sklearn.model_selection import GridSearchCV
    num_neurons = [[5, 10], [5, 10, 20]]
    class_weight=class_weight = [{0:x, 1:1-x} for x in np.linspace(0.1, 0.5, 2)]
    param_grid = dict(num_neurons=num_neurons,
                      class_weight=class_weight)

    model = KerasClassifier(build_fn=create_model, input_dim=X_embed.shape[-1], epochs=10, verbose=0)
    g = GridSearchCV(estimator=model, param_grid=param_grid, verbose=1, scoring='recall')
    r = g.fit(X_embed, y_train)
    
    
    # Wide and best deep
    
    # we need to create separate sequential models for each embedding
    embed_branches = []
    X_ints_train = []
    X_ints_test = []
    all_inputs = []
    all_branch_outputs = []

    for cols in cross_columns:
        # encode crossed columns as ints for the embedding
        enc = LabelEncoder()

        # create crossed labels
        # needs to be commented better, Eric!
        X_crossed_train = df_train[cols].apply(lambda x: '_'.join(x), axis=1)
        X_crossed_test = df_test[cols].apply(lambda x: '_'.join(x), axis=1)

        enc.fit(np.hstack((X_crossed_train.values,  X_crossed_test.values)))
        X_crossed_train = enc.transform(X_crossed_train)
        X_crossed_test = enc.transform(X_crossed_test)
        X_ints_train.append( X_crossed_train )
        X_ints_test.append( X_crossed_test )

        # get the number of categories
        N = max(X_ints_train[-1]+1) # same as the max(df_train[col])

        # create embedding branch from the number of categories
        inputs = Input(shape=(1,),dtype='int32', name = '_'.join(cols))
        all_inputs.append(inputs)
        x = Embedding(input_dim=N, output_dim=int(np.sqrt(N)), input_length=1)(inputs)
        x = Flatten()(x)
        all_branch_outputs.append(x)

    # merge the branches together
    wide_branch = concatenate(all_branch_outputs)

    # reset this input branch
    all_branch_outputs = []
    # add in the embeddings
    for col in categorical_features:
        # encode as ints for the embedding
        X_ints_train.append( df_train[col].values )
        X_ints_test.append( df_test[col].values )

        # get the number of categories
        N = max(X_ints_train[-1]+1) # same as the max(df_train[col])

        # create embedding branch from the number of categories
        inputs = Input(shape=(1,),dtype='int32', name=col)
        all_inputs.append(inputs)
        x = Embedding(input_dim=N, output_dim=int(np.sqrt(N)), input_length=1)(inputs)
        x = Flatten()(x)
        all_branch_outputs.append(x)

    # also get a dense branch of the numeric features
    all_inputs.append(Input(shape=(X_train_num.shape[1],),sparse=False,name='numeric_data'))
    x = Dense(units=20, activation='relu')(all_inputs[-1])
    all_branch_outputs.append( x )

    # merge the branches together
    deep_branch = concatenate(all_branch_outputs)

    # here is where we'll use the result of the GridSearch
    for layer in best_deep.layers[:-1]:
        deep_branch = layer(deep_branch)

    final_branch = concatenate([wide_branch, deep_branch])
    final_branch = Dense(units=1,activation='sigmoid')(final_branch)

    model = Model(inputs=all_inputs, outputs=final_branch)

    model.compile(optimizer='adagrad',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    print('fitting the whole thing')
    model.fit(X_ints_train+ [X_train_num],
            y_train, epochs=10, batch_size=32, verbose=1, class_weight=best_deep_params['class_weight'])
    
    yhat = np.round(model.predict(X_ints_test + [X_test_num]))
    score = mt.recall_score(y_test,yhat)
    print(score)
    scores.append(score)

C:\Users\Cameron\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:444: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


training embedings
Epoch 1/10
1175/1175 [==============================] - 2s - loss: 0.1654 - acc: 0.8323     
Epoch 2/10
1175/1175 [==============================] - 0s - loss: 0.1404 - acc: 0.8391     
Epoch 3/10
1175/1175 [==============================] - 0s - loss: 0.1326 - acc: 0.8391     
Epoch 4/10
1175/1175 [==============================] - 0s - loss: 0.1272 - acc: 0.8391     
Epoch 5/10
1175/1175 [==============================] - 0s - loss: 0.1225 - acc: 0.8391     
Epoch 6/10
1175/1175 [==============================] - 0s - loss: 0.1183 - acc: 0.8391     
Epoch 7/10
1175/1175 [==============================] - 0s - loss: 0.1143 - acc: 0.8391     
Epoch 8/10
1175/1175 [==============================] - 0s - loss: 0.1106 - acc: 0.8400     
Epoch 9/10
1175/1175 [==============================] - 0s - loss: 0.1071 - acc: 0.8409     
Epoch 10/10
1175/1175 [==============================] - 0s - loss: 0.1038 - acc: 0.8477     
Grid Searching the layers of the Deep
Fitting 3 fo

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:  1.1min finished


fitting the whole thing
Epoch 1/10
1175/1175 [==============================] - 2s - loss: 0.0733 - acc: 0.8417     
Epoch 2/10
1175/1175 [==============================] - 0s - loss: 0.0190 - acc: 1.0000     
Epoch 3/10
1175/1175 [==============================] - 0s - loss: 0.0088 - acc: 1.0000     
Epoch 4/10
1175/1175 [==============================] - 0s - loss: 0.0054 - acc: 1.0000     
Epoch 5/10
1175/1175 [==============================] - 0s - loss: 0.0037 - acc: 1.0000     
Epoch 6/10
1175/1175 [==============================] - 0s - loss: 0.0028 - acc: 1.0000     
Epoch 7/10
1175/1175 [==============================] - 0s - loss: 0.0022 - acc: 1.0000     
Epoch 8/10
1175/1175 [==============================] - 0s - loss: 0.0018 - acc: 1.0000     
Epoch 9/10
1175/1175 [==============================] - 0s - loss: 0.0015 - acc: 1.0000     
Epoch 10/10
1175/1175 [==============================] - 0s - loss: 0.0013 - acc: 1.0000     
1.0
training embedings
Epoch 1/10
1175/1175 [

In [99]:
type(X.values)

numpy.ndarray